In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from transformers import SwinForImageClassification, AutoImageProcessor

import wandb

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

## Define Environment Variables

In [2]:
%env WANDB_PROJECT=WallPaperDefectTypeClassification
%env WANDB_NOTEBOOK_NAME=./inference.ipynb
%env WANDB_RUN_GROUP=exp2
%env WANDB_JOB_TYPE=infer

env: WANDB_PROJECT=WallPaperDefectTypeClassification
env: WANDB_NOTEBOOK_NAME=./inference.ipynb
env: WANDB_RUN_GROUP=exp2
env: WANDB_JOB_TYPE=infer


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device.type

'cuda'

## Hyperparameter Setting

In [4]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':6,
    'BATCH_SIZE':32,
    'SEED':42,
    'NUM_WORKERS':4,
    'MODEL_DIR': "2gnldud/WallPaperDefectTypeClassification/model-pretty-sweep-1:v0",
    'MODEL_VER' : "0.0.3_base",
}

In [5]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None, processor=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.processor = processor

    def __getitem__(self, index):
        
        img_path = self.img_path_list[index]
        image = Image.open(img_path)
        image_tr = self.transforms(image=np.array(image))['image']
        pixel_values = self.processor(image_tr, return_tensors="pt").pixel_values.squeeze()
        
        if self.label_list is not None:
            label = self.label_list[index]
            return {
                'pixel_values': pixel_values, 
                'label': label,
                }
        else:
            return {
                'pixel_values': pixel_values,
                }
        
    def __len__(self):
        return len(self.img_path_list)

In [6]:
test_transform = A.Compose([
                            ToTensorV2()
                            ])

In [7]:
run = wandb.init()
artifact = run.use_artifact(CFG['MODEL_DIR'], type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: 2gnldud. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-pretty-sweep-1:v0, 331.57MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.4


In [8]:
model = SwinForImageClassification.from_pretrained(artifact_dir).to(device)
image_processor = AutoImageProcessor.from_pretrained(artifact_dir)

## Inference

In [9]:
test = pd.read_csv('../data/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: str(x).replace("./", "../data/"))

In [10]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform, image_processor)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [11]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for inputs in tqdm(iter(test_loader)):
            inputs['pixel_values'] = inputs['pixel_values'].to(device)
            logits = model(**inputs).logits
            predicted_label = torch.argmax(logits, dim=-1).tolist()
            
            preds += [model.config.id2label[x] for x in predicted_label]
            
    return preds

In [12]:
preds = inference(model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [13]:
submit = pd.read_csv('../data/sample_submission.csv')

In [14]:
submit['label'] = preds

In [15]:
submit.to_csv(f"../submissions/submit_{CFG['MODEL_DIR'].split('/')[-1]}.csv", index=False)